train, test, val data를 코드를 이용하여 구분합니다.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# ========================================================================================
# [설정] 파일 경로 및 비율
# ========================================================================================
CONFIG = {
    'input_file': '통합_리뷰_기업중복제거.xlsx - Sheet1.csv', # [수정] 업로드된 CSV 파일명으로 변경
    'train_ratio': 0.8,
    'val_ratio': 0.1,
    'test_ratio': 0.1,
    'seed': 42 # 랜덤 시드 고정 (언제 실행해도 똑같이 섞이도록)
}

def split_and_save_dataset():
    # 1. 원본 데이터 로드
    if not os.path.exists(CONFIG['input_file']):
        print(f"❌ 오류: '{CONFIG['input_file']}' 파일을 찾을 수 없습니다.")
        print("   (코랩의 좌측 파일 탭에 파일을 업로드했는지 확인해주세요.)")
        return

    print(f"📂 원본 데이터 로딩 중... ({CONFIG['input_file']})")

    try:
        # [수정] read_excel -> read_csv 변경 (CSV 파일이므로)
        df = pd.read_csv(CONFIG['input_file'])
    except Exception as e:
        print(f"❌ 파일을 읽는 도중 오류가 발생했습니다: {e}")
        return

    total_len = len(df)
    print(f" -> 전체 데이터 수: {total_len}개")

    # 2. 데이터 셔플 및 분할 (8:1:1)
    # 먼저 Train(80%) vs Temp(20%)로 나눕니다.
    train_df, temp_df = train_test_split(
        df,
        test_size=(1 - CONFIG['train_ratio']),
        random_state=CONFIG['seed'],
        shuffle=True # 무작위 섞기 필수
    )

    # 남은 Temp(20%)를 절반으로 나누어 Val(10%) vs Test(10%)로 만듭니다.
    # (temp_df 내에서의 0.5는 전체의 0.1이 됩니다)
    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.5,
        random_state=CONFIG['seed'],
        shuffle=True
    )

    # 3. 분할 결과 확인
    print("\n📊 데이터 분할 결과")
    print("-" * 30)
    print(f"1️⃣ Train Set (학습) : {len(train_df)}개 ({len(train_df)/total_len*100:.1f}%)")
    print(f"2️⃣ Valid Set (검증) : {len(val_df)}개 ({len(val_df)/total_len*100:.1f}%)")
    print(f"3️⃣ Test Set  (평가) : {len(test_df)}개 ({len(test_df)/total_len*100:.1f}%)")
    print("-" * 30)
    print(f"합계 : {len(train_df) + len(val_df) + len(test_df)}개")

    # 4. 파일로 저장
    print("\n💾 파일 저장 중...")
    # 학습/평가용 데이터는 호환성이 좋은 CSV로 저장합니다.
    train_df.to_csv('train.csv', index=False, encoding='utf-8-sig')
    val_df.to_csv('val.csv', index=False, encoding='utf-8-sig')
    test_df.to_csv('test.csv', index=False, encoding='utf-8-sig')

    print("✅ 저장 완료! 다음 파일들이 생성되었습니다:")
    print("   - train.csv")
    print("   - val.csv")
    print("   - test.csv")

if __name__ == "__main__":
    split_and_save_dataset()

구분한 데이터셋을 이용하여 모델을 학습 및 평가의 과정을 거쳐 성능이 가장 좋은 모델의 체크포인트를 확인하도록 합니다.

In [ ]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
import numpy as np
import pandas as pd # pandas 추가
from torch.utils.data import Dataset, DataLoader # Dataset, DataLoader 추가
from sklearn.metrics import mean_absolute_error, r2_score
import os

# ========================================================================================
# [Data Loader Section] 데이터 로드 및 전처리 클래스 (통합됨)
# ========================================================================================

# 데이터 설정 (data_loader.py 내용을 여기로 가져옴)
DATA_CONFIG = {
    'model_name': 'klue/bert-base',
    'max_len': 128,
    'batch_size': 16,
    'train_file': 'train.csv',
    'val_file': 'val.csv',
    'test_file': 'test.csv',
}

class ReviewDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'review_text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.float)
        }

def preprocess_dataframe(df):
    """CSV 데이터 전처리 함수"""
    df.columns = [str(col).strip() for col in df.columns]

    rename_map = {
        '구분': 'job_role', '기업명': 'company',
        '복지 및 급여': 'salary', '업무와 삶의 균형': 'wlb', '사내문화': 'culture',
        '경영진': 'leadership', '승진 기회': 'promotion', '종합평점': 'overall'
    }
    available_cols = set(df.columns)
    valid_rename = {k:v for k,v in rename_map.items() if k in available_cols}
    df = df.rename(columns=valid_rename)

    df['full_text'] = (
        df['장점'].fillna('').astype(str) + " " +
        df['단점'].fillna('').astype(str) + " " +
        df['요약'].fillna('').astype(str)
    )

    target_cols = ['salary', 'wlb', 'culture', 'leadership', 'promotion', 'overall']
    for col in target_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(3.0)

    valid_target_cols = [col for col in target_cols if col in df.columns]
    return df, valid_target_cols

def get_data_loaders():
    """데이터셋 파일을 로드하여 DataLoader 생성"""
    print("\n[Data Loader] CSV 파일 로드 시작...")

    # 파일 경로 확인
    files = {
        'Train': DATA_CONFIG['train_file'],
        'Val': DATA_CONFIG['val_file'],
        'Test': DATA_CONFIG['test_file']
    }

    for name, path in files.items():
        if not os.path.exists(path):
            raise FileNotFoundError(f"❌ '{path}' 파일이 없습니다. dataset_splitter.py를 먼저 실행하거나 파일을 업로드해주세요.")

    df_train = pd.read_csv(files['Train'])
    df_val = pd.read_csv(files['Val'])
    df_test = pd.read_csv(files['Test'])

    df_train, target_cols = preprocess_dataframe(df_train)
    df_val, _ = preprocess_dataframe(df_val)
    df_test, _ = preprocess_dataframe(df_test)

    tokenizer = AutoTokenizer.from_pretrained(DATA_CONFIG['model_name'])

    def create_loader(df):
        ds = ReviewDataset(
            texts=df['full_text'].to_numpy(),
            targets=df[target_cols].to_numpy(),
            tokenizer=tokenizer,
            max_len=DATA_CONFIG['max_len']
        )
        return DataLoader(ds, batch_size=DATA_CONFIG['batch_size'], num_workers=0)

    return create_loader(df_train), create_loader(df_val), create_loader(df_test), target_cols, tokenizer


# ========================================================================================
# [설정] 학습 하이퍼파라미터
# ========================================================================================
TRAIN_CONFIG = {
    'epochs': 5,             # 학습 반복 횟수
    'learning_rate': 2e-5,   # 학습률
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'model_save_path': 'review_rating_model.pt' # 학습된 모델 저장 경로
}

# ========================================================================================
# [Model Architecture] BERT Regressor
# ========================================================================================
class ReviewRatingRegressor(nn.Module):
    def __init__(self, n_targets=6):
        super(ReviewRatingRegressor, self).__init__()
        # DATA_CONFIG를 직접 참조
        self.bert = AutoModel.from_pretrained(DATA_CONFIG['model_name'])
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_targets)

    def forward(self, input_ids, attention_mask):
        _, pooler_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        output = self.drop(pooler_output)
        return self.out(output), pooler_output

# ========================================================================================
# [Training & Evaluation Steps]
# ========================================================================================
def train_step(model, data_loader, loss_fn, optimizer, scheduler, device):
    model = model.train()
    losses = []

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs, _ = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)

        losses.append(loss.item())
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return np.mean(losses)

def eval_step(model, data_loader, loss_fn, device):
    model = model.eval()
    losses = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs, _ = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

    return np.mean(losses)

# ========================================================================================
# [Main Execution Flow]
# ========================================================================================
def run_training():
    print("="*60)
    print("🚀 리뷰 평점 예측 모델 학습 시작")
    print("="*60)

    # 1. 데이터 로드 (내부 함수 호출)
    train_loader, val_loader, test_loader, target_cols, _ = get_data_loaders()

    # 2. 모델 초기화
    device = TRAIN_CONFIG['device']
    print(f"\n[Setup] Device: {device}")
    model = ReviewRatingRegressor(n_targets=len(target_cols)).to(device)

    # 3. Optimizer & Scheduler 설정
    optimizer = AdamW(model.parameters(), lr=TRAIN_CONFIG['learning_rate'])
    total_steps = len(train_loader) * TRAIN_CONFIG['epochs']
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    loss_fn = nn.MSELoss().to(device)

    # 4. 학습 루프 (Training Loop)
    print("\n[Training Process]")
    best_loss = float('inf')

    for epoch in range(TRAIN_CONFIG['epochs']):
        train_loss = train_step(model, train_loader, loss_fn, optimizer, scheduler, device)
        val_loss = eval_step(model, val_loader, loss_fn, device)

        print(f"Epoch {epoch+1}/{TRAIN_CONFIG['epochs']} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        if val_loss < best_loss:
            torch.save(model.state_dict(), TRAIN_CONFIG['model_save_path'])
            best_loss = val_loss
            print(f"   ✅ Best Model Saved! (Val Loss: {best_loss:.4f})")

    # 5. 최종 평가 (Test Set Evaluation)
    print("\n" + "="*60)
    print("🏆 최종 모델 평가 (Test Set)")
    print("="*60)

    if os.path.exists(TRAIN_CONFIG['model_save_path']):
        model.load_state_dict(torch.load(TRAIN_CONFIG['model_save_path']))
        print(" -> 학습 중 저장된 최고의 모델 가중치를 로드했습니다.")
    else:
        print(" -> 저장된 모델이 없어 마지막 에포크 모델로 평가합니다.")

    model.eval()

    preds, reals = [], []
    with torch.no_grad():
        for d in test_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs, _ = model(input_ids=input_ids, attention_mask=attention_mask)

            preds.extend(outputs.cpu().numpy())
            reals.extend(targets.cpu().numpy())

    preds = np.array(preds)
    reals = np.array(reals)

    mae = mean_absolute_error(reals, preds)
    r2 = r2_score(reals, preds)

    print(f"\n📊 Test 결과 요약:")
    print(f"   - MAE (평균 오차) : {mae:.4f} (낮을수록 좋음)")
    print(f"   - R2 Score (결정계수): {r2:.4f} (1에 가까울수록 좋음)")

    print("\n[💡 해석]")
    print(f"   이 모델은 사용자의 리뷰 텍스트만 보고 실제 평점과 평균 {mae:.2f}점 차이로 예측하고 있습니다.")

if __name__ == "__main__":
    run_training()